In [27]:
import pandas as pd

data = pd.read_csv("crop_yield.csv")

# Define relevant columns
categorical_cols = ['Region', 'Soil_Type', 'Weather_Condition']
continuous_cols = ['Rainfall_mm', 'Temperature_Celsius']
target_col = 'Crop'

n_samples = 1000

combined_samples = pd.DataFrame()

for crop, group in data.groupby(target_col):
    # Sort by yield
    sorted_group = group.sort_values(by='Yield_tons_per_hectare', ascending=False)
    
    top_group = sorted_group.head(n_samples)
    top_group['Yield_Category'] = 'Top'

    middle_start = len(sorted_group) // 2 - n_samples // 2
    middle_group = sorted_group.iloc[middle_start:middle_start + n_samples]
    middle_group['Yield_Category'] = 'Middle'

    combined_samples = pd.concat([combined_samples, top_group, middle_group])

combined_samples = combined_samples[categorical_cols + continuous_cols + [target_col, 'Yield_tons_per_hectare', 'Yield_Category']]

# Save to a new CSV file
combined_samples.to_csv("combined_yield_samples_unencoded.csv", index=False)

print("New 'combined_yield_samples_unencoded.csv' created with top and middle yield samples per crop.")
print(combined_samples.head())

/tmp/ipykernel_69/1467659076.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_group['Yield_Category'] = 'Top'
/tmp/ipykernel_69/1467659076.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  middle_group['Yield_Category'] = 'Middle'
/tmp/ipykernel_69/1467659076.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

New 'combined_yield_samples_unencoded.csv' created with top and middle yield samples per crop.
       Region Soil_Type Weather_Condition  Rainfall_mm  Temperature_Celsius  \
905016  North    Chalky            Cloudy   874.775592            27.909656   
572954   East      Loam            Cloudy   933.096993            39.445050   
461357   West      Clay             Rainy   996.078325            36.972621   
350006   West      Loam            Cloudy   910.598704            38.842505   
224117   West      Silt             Rainy   954.911259            34.653132   

          Crop  Yield_tons_per_hectare Yield_Category  
905016  Barley                9.949207            Top  
572954  Barley                9.744646            Top  
461357  Barley                9.595395            Top  
350006  Barley                9.580710            Top  
224117  Barley                9.580166            Top  


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pennylane as qml
from sklearn.model_selection import train_test_split
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.filterwarnings("ignore", message="X does not have valid feature names, but MinMaxScaler was fitted with feature names")

# Load the dataset
data = pd.read_csv("combined_yield_samples_unencoded.csv")

categorical_cols = ['Region', 'Soil_Type', 'Weather_Condition']
continuous_cols = ['Rainfall_mm', 'Temperature_Celsius']
target_col = 'Crop'

# Define crop conditions and icons
crop_conditions = {
    'Barley': {'min_rainfall': 350, 'max_rainfall': 850, 'min_temperature': -8, 'max_temperature': 30, 'soil_types': ['Loam', 'Clay', 'Sandy', 'Silt', 'Chalky']},
    'Rice': {'min_rainfall': 900, 'max_rainfall': 2200, 'min_temperature': 18, 'max_temperature': 40, 'soil_types': ['Loam', 'Clay', 'Peaty', 'Silt']},
    'Wheat': {'min_rainfall': 250, 'max_rainfall': 1000, 'min_temperature': 0, 'max_temperature': 34, 'soil_types': ['Loam', 'Clay', 'Sandy', 'Silt', 'Chalky']},
    'Cotton': {'min_rainfall': 450, 'max_rainfall': 1600, 'min_temperature': 15, 'max_temperature': 37, 'soil_types': ['Loam', 'Sandy', 'Clay', 'Chalky']},
    'Soybean': {'min_rainfall': 400, 'max_rainfall': 950, 'min_temperature': 15, 'max_temperature': 32, 'soil_types': ['Loam', 'Clay', 'Silt', 'Peaty']},
    'Maize': {'min_rainfall': 500, 'max_rainfall': 1300, 'min_temperature': 10, 'max_temperature': 35, 'soil_types': ['Loam', 'Clay', 'Sandy', 'Silt']},
}

# Icon paths
crop_icons = {
    'Barley': 'icons/barley.png',
    'Rice': 'icons/rice.png',
    'Wheat': 'icons/wheat.png',
    'Cotton': 'icons/cotton.png',
    'Soybean': 'icons/soybean.png',
    'Maize': 'icons/maize.png'
}

# Encode categorical features
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Normalize continuous features
scaler = MinMaxScaler()
data[continuous_cols] = scaler.fit_transform(data[continuous_cols])

X = data[categorical_cols + continuous_cols]
y = data[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Quantum setup
n_qubits = X.shape[1]
dev = qml.device("default.qubit", wires=n_qubits)

# Initialize parameters for feature map and variational layer
feature_params = np.random.rand(n_qubits)
variational_params = np.random.rand(n_qubits)

# Define the entangled feature map with parameterization
def feature_map(x, params, weight_rainfall=5, weight_temperature=2):
    for i in range(len(categorical_cols)):
        qml.RY(np.pi * x[i] * params[i], wires=i)
    qml.RY(np.pi * x[len(categorical_cols)] * weight_rainfall * params[len(categorical_cols)], wires=len(categorical_cols))
    qml.RY(np.pi * x[len(categorical_cols) + 1] * weight_temperature * params[len(categorical_cols) + 1], wires=len(categorical_cols) + 1)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

# Define the variational layer
def variational_layer(params):
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
        if i < n_qubits - 1:
            qml.CNOT(wires=[i, i + 1])

# Define the quantum kernel function
def quantum_kernel(x1, x2):
    @qml.qnode(dev)
    def kernel_circuit():
        feature_map(x1, feature_params)
        variational_layer(variational_params)
        qml.adjoint(feature_map)(x2, feature_params)
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    return sum(kernel_circuit())

region_dropdown = widgets.Dropdown(options=label_encoders['Region'].classes_, description='Region:')
soil_dropdown = widgets.Dropdown(options=label_encoders['Soil_Type'].classes_, description='Soil Type:')
weather_dropdown = widgets.Dropdown(options=label_encoders['Weather_Condition'].classes_, description='Weather Condition:')
rainfall_slider = widgets.IntSlider(min=0, max=2000, step=50, value=1000, description='Rainfall (mm):')
temperature_slider = widgets.IntSlider(min=0, max=50, step=1, value=28, description='Temperature (°C):')
output = widgets.Output()

def get_recommendations(region, soil, weather, rainfall, temperature):
    output.clear_output()
    
    with output:
        display(widgets.Label("Calculating recommendations, please wait..."))

        default_input = {
            'Region': label_encoders['Region'].transform([region])[0],
            'Soil_Type': label_encoders['Soil_Type'].transform([soil])[0],
            'Weather_Condition': label_encoders['Weather_Condition'].transform([weather])[0]
        }

        # Scale continuous features
        continuous_placeholder = scaler.transform([[rainfall, temperature]])
        scaled_continuous = continuous_placeholder[0]

        input_data = np.concatenate([list(default_input.values()), scaled_continuous])

        recommended_crops = {}
        non_recommended_crops = {}

        for crop, conditions in crop_conditions.items():
            # Check if the crop meets all conditions
            if (rainfall >= conditions['min_rainfall'] and 
                rainfall <= conditions['max_rainfall'] and
                temperature >= conditions['min_temperature'] and
                temperature <= conditions['max_temperature'] and
                soil in conditions['soil_types']):
                
                # Calculate similarity if crop is recommended
                similarity = np.mean([quantum_kernel(input_data, X_train.iloc[i].values) 
                                      for i in range(len(X_train)) if y_train.iloc[i] == crop])
                recommended_crops[crop] = similarity
            else:
                # Record why the crop is not recommended
                reasons = []
                if rainfall < conditions['min_rainfall']:
                    reasons.append(f"Minimum rainfall: {conditions['min_rainfall']} mm (Input: {rainfall} mm)")
                if rainfall > conditions['max_rainfall']:
                    reasons.append(f"Maximum rainfall: {conditions['max_rainfall']} mm (Input: {rainfall} mm)")
                if temperature < conditions['min_temperature']:
                    reasons.append(f"Minimum temperature: {conditions['min_temperature']} °C (Input: {temperature} °C)")
                if temperature > conditions['max_temperature']:
                    reasons.append(f"Maximum temperature: {conditions['max_temperature']} °C (Input: {temperature} °C)")
                if soil not in conditions['soil_types']:
                    reasons.append(f"Suitable soils: {', '.join(conditions['soil_types'])} (Input: {soil})")
                non_recommended_crops[crop] = reasons

        crop_html = ""
        for crop, icon in crop_icons.items():
            border_color = "green" if crop in recommended_crops else "red"
            crop_html += f"""
            <div style="display: inline-block; text-align: center; margin: 10px;">
                <img src="{icon}" alt="{crop}" style="width: 100px; height: 100px; border: 5px solid {border_color}; border-radius: 10px;">
                <div>{crop}</div>
            </div>
            """

        output.clear_output()
        display(HTML(f"<div style='display: flex; flex-wrap: wrap;'>{crop_html}</div>"))

        print("\nRecommended crops for the selected conditions:")
        for crop, similarity in sorted(recommended_crops.items(), key=lambda x: x[1], reverse=True):
            print(f"- {crop} (Average similarity score: {similarity})")

        print("\nCrops that do not meet the conditions and why:")
        for crop, reasons in non_recommended_crops.items():
            print(f"- {crop}:")
            for reason in reasons:
                print(f"  - {reason}")

recommend_button = widgets.Button(description="Get Recommendations")

def on_button_click(b):
    get_recommendations(
        region=region_dropdown.value,
        soil=soil_dropdown.value,
        weather=weather_dropdown.value,
        rainfall=rainfall_slider.value,
        temperature=temperature_slider.value
    )

recommend_button.on_click(on_button_click)

display(region_dropdown, soil_dropdown, weather_dropdown, rainfall_slider, temperature_slider, recommend_button, output)

Dropdown(description='Region:', options=('East', 'North', 'South', 'West'), value='East')

Dropdown(description='Soil Type:', options=('Chalky', 'Clay', 'Loam', 'Peaty', 'Sandy', 'Silt'), value='Chalky…

Dropdown(description='Weather Condition:', options=('Cloudy', 'Rainy', 'Sunny'), value='Cloudy')

IntSlider(value=1000, description='Rainfall (mm):', max=2000, step=50)

IntSlider(value=28, description='Temperature (°C):', max=50)

Button(description='Get Recommendations', style=ButtonStyle())

Output()